In [ ]:
def load_data(f):    
  d = {}
  count = 0
  d[count] = [] 
  for i, line in enumerate(f):  
    if i % 8 == 0 and i != 0:
      count +=1    
      d[count] = []
    line = line.strip('[REG]').strip()
    if line == '<NA>':
      line = np.nan  
    d[count].append(line)
  f.close()
  df = pd.DataFrame.from_dict(d, orient='index', columns = ['Registro', 'id_viaje', 'duracion_estadia', 'genero', 'edad', 'ninos', 'codigo_destino', 'tipo_acomodacion'])  
  df.drop(['Registro'], axis = 1, inplace = True)
  df.drop(index=9868, inplace = True)
  return df

In [ ]:
def dataTypeTransform(df):
  df['id_viaje'] = df['id_viaje'].astype(int)
  df['duracion_estadia'] = df['duracion_estadia'].astype(int)  
  df['edad'] = pd.to_numeric(df['edad'], downcast = 'integer', errors='coerce')
  df['genero'] = df['genero'].astype('category')
  df['ninos'] = df['ninos'].astype(int, errors = 'ignore')
  df = df.convert_dtypes()
  return df

In [ ]:
def freq_table(df, cols):    
    for col in cols:        
        freq_tables = df[col].value_counts().to_frame().reset_index()
        freq_tables['frequency']= freq_tables[col].apply(lambda x: x/df.shape[0])
        print("="*40)
        print(freq_tables)

In [ ]:
def plotBarPerColumns(df, cols, nGraphPerRow):
    nGraphRow = int(len(cols) / nGraphPerRow)
    nGraphPerRow = int(nGraphPerRow)    
    plt.figure(num = None, figsize = (3 * nGraphPerRow, 4 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')  
    for i, col in enumerate(cols):
        plt.subplot(nGraphRow, nGraphPerRow, i+1)          
        valueCounts = df[col].value_counts()
        valueCounts.plot.bar()        
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{col} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
def plotHistPerColumns(df, cols, nGraphPerRow):
    nGraphRow = int(len(cols) / nGraphPerRow)
    nGraphPerRow = int(nGraphPerRow) 
    plt.figure(num = None, figsize = (3 * nGraphPerRow, 4 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i, col in enumerate(cols):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)             
        df[col].hist()        
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{col} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [1]:
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include = [np.number])
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]]
    columnsNames = list(df)
    if len(columnsNames) > 10:
        columnsNames = columnsNames[:10]
    df = df[columnsNames]
    ax = pd.plotting.scatter_matrix(df, alpha = 0.75, figsize = [plotSize, plotSize], diagonal = 'kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr, coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords = 'axes fraction', ha = 'center', va = 'center', size = textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()